In [1]:
# dependencies
import json
import pandas as pd
import re

# load types file
with open('../API_Data/pkmn.json', 'r') as in_file:
    pkmn = json.load(in_file)

In [118]:
pkmn_names = pd.Series([pkmn[num]['name'] for num in range(len(pkmn))])

Here I'm going to want to change out form names to something that looks a little better (ex: raichu-alola -> Alolan Raichu)

In [119]:
def clean_names(name):
    
    # drop gigantimaxes, pikachu and eevee costumes, totems
    to_drop = ['gmax', 'pikachu-', 'eevee-', 'totem']
    for pattern in to_drop:
        if re.match(f'.*{pattern}', name, flags = re.IGNORECASE):
            name = 'DROP'
            return name

    # capture all region-specific forms
    regions = ['alol', 'galar', 'hisui']
    for region in regions:
        if re.match(f'.*{region}', name, flags=re.IGNORECASE):
            
            new = re.search(f'(.*)-{region}', name)
            if new:
                if region in ['alol', 'hisui']:
                    name = f'{region}an {new.group(1)}'
                elif region in ['galar']:
                    name = f'{region}ian {new.group(1)}'
            return name
    
    # pokemon with different formes
    forme_search = ['incarnate', 'therian', 'normal', 'attack', 'defense', 'speed', 'origin', 'altered', 'land', 'aria', 'pirouette', 'shield', 'blade', 'sky']
    for pattern in forme_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} ({pattern} forme)'
            return name
    
    # pokemon with different forms
    form_search = ['sunny', 'rainy', 'snowy', 'ordinary', 'resolute', 'midday', 'midnight', 'dusk', 'solo', 'school', 'amped', 'low-key', 'dawn', 'ultra']
    for pattern in form_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} ({pattern} form)'
            return name
    
    # replace dash with space
    poke_search = ['tapu', 'zacian', 'zamazenta', 'urshifu', 'calyrex', 'morpeko', 'pumpkaboo', 'gourgeist', 'oricorio']
    for pattern in poke_search:
        if re.match(f'{pattern}-.*', name, flags = re.IGNORECASE):
            new = re.search(f'{pattern}-(.*)', name)
            name = f'{pattern} {new.group(1).replace("-", " ")}'
            return name
    
    # pokemon with different genders
    gender_search = ['m', 'male', 'f', 'female']
    for pattern in gender_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} ({pattern})'
            return name
    
    # pokemon with different cloaks (burmy and wormadam)
    cloak_search = ['plant', 'sandy', 'trash']
    for pattern in cloak_search:
        if re.match(f'.*-{pattern}$', name, flags = re.IGNORECASE):
            new = re.search(f'(.*)-{pattern}', name)
            name = f'{new.group(1)} ({pattern} cloak)'
            return name
    
    # rotom
    if re.match(f'rotom-.*', name, flags = re.IGNORECASE):
        new = re.search(f'rotom-(.*)', name)
        name = f'{new.group(1)} rotom'
        return name

    # minior (just keep one color since they're all the same)
    if re.match(f'minior-.*', name, flags = re.IGNORECASE):
        new = re.search(f'minior-(.*)', name)
        if new.group(1) == 'red':
            name = f'minior (core form)'
        elif new.group(1) == 'red-meteor':
            name = f'minior (meteor form)'
        else:
            name = 'DROP'
        return name
    
    # capture all mega evolutions
    if re.match('.*mega', name, flags=re.IGNORECASE):
        
        if re.match('.*-mega$', name, flags=re.IGNORECASE):
            new = re.search(r'(.*)-mega', name)
            if new:
                name = 'mega ' + new.group(1)
        elif re.match('.*-mega-[xy]', name, flags=re.IGNORECASE):
            new = re.search(r'(.*)-mega-([xy])', name)
            if new:
                name = 'mega ' + new.group(1) + ' ' + new.group(2)
            return name
        
    return name


pkmn_names = pkmn_names.apply(clean_names)

In [124]:
for name in pkmn_names[pkmn_names.str.contains('-')]:
    print(name)

mr-mime
ho-oh
mime-jr
porygon-z
basculin-red-striped
darmanitan-standard
zygarde-50
type-null
mimikyu-disguised
jangmo-o
hakamo-o
kommo-o
mr-rime
eiscue-ice
basculin-blue-striped
darmanitan-zen
kyurem-black
kyurem-white
floette-eternal
kyogre-primal
groudon-primal
hoopa-unbound
greninja-battle-bond
greninja-ash
zygarde-10-power-construct
zygarde-50-power-construct
zygarde-complete
mimikyu-busted
magearna-original
rockruff-own-tempo
galarian mr-mime
zygarde-10
cramorant-gulping
cramorant-gorging
toxtricity (low-key form)
eiscue-noice
eternatus-eternamax
zarude-dada
basculin-white-striped


Pokemon names to leave alone:
Ho-Oh
Jangmo-o
Hakamo-o
Kommo-o

Single edits:
fix Darmanitan, mr mime
type-null -> Type: Null
Zygarde (forme)
Basculin (form) (add striped forms and replace - with a space?)
Toxtricity Low-Key (form)
Mimikyu?

replace - with space and () following:
cramorant
eiscue
kyurem
calyrex (move from plain space)

all the form/forme searches: move those to searching for the pokemon name instead of the form? more future-proofed